In [1]:
import numpy as np
import pandas as pd
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, fpmax, fpgrowth 

In [2]:
ds = pd.read_csv('Online Retail.csv')

In [3]:
ds

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12-1-2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12-1-2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12-1-2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12-1-2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12-1-2010 8:26,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12-9-2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12-9-2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12-9-2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12-9-2011 12:50,4.15,12680.0,France


In [4]:
#We will explore data from the country of France
ds = ds.query("Country == 'France'")
ds

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
26,536370,22728,ALARM CLOCK BAKELIKE PINK,24,12-1-2010 8:45,3.75,12583.0,France
27,536370,22727,ALARM CLOCK BAKELIKE RED,24,12-1-2010 8:45,3.75,12583.0,France
28,536370,22726,ALARM CLOCK BAKELIKE GREEN,12,12-1-2010 8:45,3.75,12583.0,France
29,536370,21724,PANDA AND BUNNIES STICKER SHEET,12,12-1-2010 8:45,0.85,12583.0,France
30,536370,21883,STARS GIFT TAPE,24,12-1-2010 8:45,0.65,12583.0,France
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,12-9-2011 12:50,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,12-9-2011 12:50,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,12-9-2011 12:50,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,12-9-2011 12:50,4.15,12680.0,France


In [5]:
#No missing and null data on our dataset.
print(ds.isna().sum(), "\n")

print(ds.isnull().sum())

InvoiceNo       0
StockCode       0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID     66
Country         0
dtype: int64 

InvoiceNo       0
StockCode       0
Description     0
Quantity        0
InvoiceDate     0
UnitPrice       0
CustomerID     66
Country         0
dtype: int64


In [6]:
#aggregate transactions based on invoice number because Same Invoice number means that it is within one Transaction
df = ds.groupby(['InvoiceNo'], as_index=False).agg(list)

In [7]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536370,"[22728, 22727, 22726, 21724, 21883, 10002, 217...","[ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI...","[24, 24, 12, 12, 24, 48, 24, 18, 24, 24, 24, 2...","[12-1-2010 8:45, 12-1-2010 8:45, 12-1-2010 8:4...","[3.75, 3.75, 3.75, 0.85, 0.65, 0.85, 1.25, 2.9...","[12583.0, 12583.0, 12583.0, 12583.0, 12583.0, ...","[France, France, France, France, France, Franc..."
1,536852,"[22549, 22544, 22539, 22661, 21791, 21786, POST]","[PICTURE DOMINOES, MINI JIGSAW SPACEBOY, MINI ...","[12, 24, 24, 10, 12, 24, 1]","[12-3-2010 9:51, 12-3-2010 9:51, 12-3-2010 9:5...","[1.45, 0.42, 0.42, 0.85, 1.25, 0.42, 18.0]","[12686.0, 12686.0, 12686.0, 12686.0, 12686.0, ...","[France, France, France, France, France, Franc..."
2,536974,"[15056BL, 15056P, 20679, 21915, 22659, 22352, ...","[EDWARDIAN PARASOL BLACK, EDWARDIAN PARASOL PI...","[9, 3, 6, 12, 12, 12, 12, 12, 2, 2, 12, 6, 10,...","[12-3-2010 13:59, 12-3-2010 13:59, 12-3-2010 1...","[5.95, 5.95, 5.95, 1.25, 1.95, 2.55, 1.25, 0.4...","[12682.0, 12682.0, 12682.0, 12682.0, 12682.0, ...","[France, France, France, France, France, Franc..."
3,537065,"[22837, 22846, 22892, 22968, 84678, 20665, 217...","[HOT WATER BOTTLE BABUSHKA , BREAD BIN DINER S...","[8, 1, 12, 8, 6, 6, 24, 24, 6, 2, 12, 24, 12, ...","[12-5-2010 11:57, 12-5-2010 11:57, 12-5-2010 1...","[4.65, 16.95, 1.25, 8.5, 2.55, 2.95, 0.42, 0.8...","[12567.0, 12567.0, 12567.0, 12567.0, 12567.0, ...","[France, France, France, France, France, Franc..."
4,537463,"[22961, 21224, 22326, 21124, 21121, 21137, 220...","[JAM MAKING SET PRINTED, SET/4 SKULL BADGES, R...","[12, 10, 18, 24, 24, 4, 6, 12, 12, 12, 4, 16, ...","[12-7-2010 10:08, 12-7-2010 10:08, 12-7-2010 1...","[1.45, 1.25, 2.95, 1.25, 1.25, 3.75, 2.55, 0.4...","[12681.0, 12681.0, 12681.0, 12681.0, 12681.0, ...","[France, France, France, France, France, Franc..."
...,...,...,...,...,...,...,...,...
456,C579532,"[POST, 22890]","[POSTAGE, NOVELTY BISCUITS CAKE STAND 3 TIER]","[-1, -4]","[11-30-2011 9:21, 11-30-2011 9:21]","[18.0, 9.95]","[12494.0, 12494.0]","[France, France]"
457,C579562,"[23084, 21731]","[RABBIT NIGHT LIGHT, RED TOADSTOOL LED NIGHT L...","[-1, -1]","[11-30-2011 11:39, 11-30-2011 11:39]","[2.08, 1.65]","[12553.0, 12553.0]","[France, France]"
458,C580161,[POST],[POSTAGE],[-2],[12-2-2011 10:49],[18.0],[12700.0],[France]
459,C580263,"[M, M, 70007, 85175, 84821, 84819, 84817, 7258...","[Manual, Manual, HI TEC ALPINE HAND WARMER, CA...","[-4, -16, -12, -16, -12, -3, -12, -36, -16, -1...","[12-2-2011 12:43, 12-2-2011 12:43, 12-2-2011 1...","[9.95, 0.29, 1.65, 0.42, 0.85, 4.25, 2.1, 0.42...","[12536.0, 12536.0, 12536.0, 12536.0, 12536.0, ...","[France, France, France, France, France, Franc..."


In [8]:
#extract only the two columns we are interested in, which are the InvoiceNo and Description.
data = df.iloc[:,[0,2]]

In [9]:
data

,InvoiceNo,Description
0,536370,"[ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI..."
1,536852,"[PICTURE DOMINOES, MINI JIGSAW SPACEBOY, MINI ..."
2,536974,"[EDWARDIAN PARASOL BLACK, EDWARDIAN PARASOL PI..."
3,537065,"[HOT WATER BOTTLE BABUSHKA , BREAD BIN DINER S..."
4,537463,"[JAM MAKING SET PRINTED, SET/4 SKULL BADGES, R..."
...,...,...
456,C579532,"[POSTAGE, NOVELTY BISCUITS CAKE STAND 3 TIER]"
457,C579562,"[RABBIT NIGHT LIGHT, RED TOADSTOOL LED NIGHT L..."
458,C580161,[POSTAGE]
459,C580263,"[Manual, Manual, HI TEC ALPINE HAND WARMER, CA..."


In [10]:
#Removing rows with invoice numbers starting with letter A and C, which indicates "cancelled" and "adjust bad debt". 
data = data[~data.iloc[:,0].str.startswith('C')]
data = data[~data.iloc[:,0].str.startswith('A')]

In [11]:
data

,InvoiceNo,Description
0,536370,"[ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI..."
1,536852,"[PICTURE DOMINOES, MINI JIGSAW SPACEBOY, MINI ..."
2,536974,"[EDWARDIAN PARASOL BLACK, EDWARDIAN PARASOL PI..."
3,537065,"[HOT WATER BOTTLE BABUSHKA , BREAD BIN DINER S..."
4,537463,"[JAM MAKING SET PRINTED, SET/4 SKULL BADGES, R..."
...,...,...
387,580986,"[LARGE RED BABUSHKA NOTEBOOK , SMALL RED BABUS..."
388,581001,"[SET OF 6 RIBBONS PERFECTLY PRETTY , MINI JIG..."
389,581171,"[TRADITIONAL PICK UP STICKS GAME , GINGERBREAD..."
390,581279,"[CAKE STAND WHITE TWO TIER LACE, REGENCY CAKES..."


In [12]:
#Checking if all Invoice No. are integer values with no letters.
pd.to_numeric(data['InvoiceNo'], errors='coerce').notnull().all()

True

In [13]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392 entries, 0 to 391
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   InvoiceNo    392 non-null    object
 1   Description  392 non-null    object
dtypes: object(2)
memory usage: 9.2+ KB


In [14]:
data.dropna()

,InvoiceNo,Description
0,536370,"[ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI..."
1,536852,"[PICTURE DOMINOES, MINI JIGSAW SPACEBOY, MINI ..."
2,536974,"[EDWARDIAN PARASOL BLACK, EDWARDIAN PARASOL PI..."
3,537065,"[HOT WATER BOTTLE BABUSHKA , BREAD BIN DINER S..."
4,537463,"[JAM MAKING SET PRINTED, SET/4 SKULL BADGES, R..."
...,...,...
387,580986,"[LARGE RED BABUSHKA NOTEBOOK , SMALL RED BABUS..."
388,581001,"[SET OF 6 RIBBONS PERFECTLY PRETTY , MINI JIG..."
389,581171,"[TRADITIONAL PICK UP STICKS GAME , GINGERBREAD..."
390,581279,"[CAKE STAND WHITE TWO TIER LACE, REGENCY CAKES..."


In [15]:
data.isna().sum()

InvoiceNo      0
Description    0
dtype: int64

In [16]:
data.isnull().sum()

InvoiceNo      0
Description    0
dtype: int64

In [17]:
data

,InvoiceNo,Description
0,536370,"[ALARM CLOCK BAKELIKE PINK, ALARM CLOCK BAKELI..."
1,536852,"[PICTURE DOMINOES, MINI JIGSAW SPACEBOY, MINI ..."
2,536974,"[EDWARDIAN PARASOL BLACK, EDWARDIAN PARASOL PI..."
3,537065,"[HOT WATER BOTTLE BABUSHKA , BREAD BIN DINER S..."
4,537463,"[JAM MAKING SET PRINTED, SET/4 SKULL BADGES, R..."
...,...,...
387,580986,"[LARGE RED BABUSHKA NOTEBOOK , SMALL RED BABUS..."
388,581001,"[SET OF 6 RIBBONS PERFECTLY PRETTY , MINI JIG..."
389,581171,"[TRADITIONAL PICK UP STICKS GAME , GINGERBREAD..."
390,581279,"[CAKE STAND WHITE TWO TIER LACE, REGENCY CAKES..."


In [18]:
#converting the Raw data into the format that is needed for the algorithm.
itemset = pd.DataFrame(data.Description.values.tolist()).add_prefix('item_')
itemset

,item_0,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8,item_9,...,item_249,item_250,item_251,item_252,item_253,item_254,item_255,item_256,item_257,item_258
0,ALARM CLOCK BAKELIKE PINK,ALARM CLOCK BAKELIKE RED,ALARM CLOCK BAKELIKE GREEN,PANDA AND BUNNIES STICKER SHEET,STARS GIFT TAPE,INFLATABLE POLITICAL GLOBE,VINTAGE HEADS AND TAILS CARD GAME,SET/2 RED RETROSPOT TEA TOWELS,ROUND SNACK BOXES SET OF4 WOODLAND,SPACEBOY LUNCH BOX,...,None,None,None,None,None,None,None,None,None,None
1,PICTURE DOMINOES,MINI JIGSAW SPACEBOY,MINI JIGSAW DOLLY GIRL,CHARLOTTE BAG DOLLY GIRL DESIGN,VINTAGE HEADS AND TAILS CARD GAME,POLKADOT RAIN HAT,POSTAGE,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,EDWARDIAN PARASOL BLACK,EDWARDIAN PARASOL PINK,EDWARDIAN PARASOL RED,RED HARMONICA IN BOX,LUNCH BOX I LOVE LONDON,LUNCH BOX WITH CUTLERY RETROSPOT,SET OF SALT AND PEPPER TOADSTOOLS,TEA PARTY BIRTHDAY CARD,ASSORTED COLOUR MINI CASES,RED RETROSPOT MINI CASES,...,None,None,None,None,None,None,None,None,None,None
3,HOT WATER BOTTLE BABUSHKA,BREAD BIN DINER STYLE RED,SET OF SALT AND PEPPER TOADSTOOLS,ROSE COTTAGE KEEPSAKE BOX,CLASSICAL ROSE SMALL VASE,RED RETROSPOT PURSE,POLKADOT RAIN HAT,RAIN PONCHO RETROSPOT,RED RETROSPOT MINI CASES,ASSORTED COLOUR MINI CASES,...,None,None,None,None,None,None,None,None,None,None
4,JAM MAKING SET PRINTED,SET/4 SKULL BADGES,ROUND SNACK BOXES SET OF4 WOODLAND,SET/10 BLUE POLKADOT PARTY CANDLES,SET/10 RED POLKADOT PARTY CANDLES,BLACK RECORD COVER FRAME,"RECORD FRAME 7"" SINGLE SIZE",ROBOT BIRTHDAY CARD,TEA PARTY BIRTHDAY CARD,"FANCY FONT BIRTHDAY CARD,",...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,LARGE RED BABUSHKA NOTEBOOK,SMALL RED BABUSHKA NOTEBOOK,LARGE PURPLE BABUSHKA NOTEBOOK,SMALL PURPLE BABUSHKA NOTEBOOK,LARGE YELLOW BABUSHKA NOTEBOOK,SMALL YELLOW BABUSHKA NOTEBOOK,SET 7 BABUSHKA NESTING BOXES,PAPERWEIGHT CHILDHOOD MEMORIES,SKULLS STORAGE BOX LARGE,SKULLS STORAGE BOX SMALL,...,None,None,None,None,None,None,None,None,None,None
388,SET OF 6 RIBBONS PERFECTLY PRETTY,MINI JIGSAW CIRCUS PARADE,MINI JIGSAW SPACEBOY,MINI LIGHTS WOODLAND MUSHROOMS,RABBIT NIGHT LIGHT,SPOTTY BUNTING,SET OF 20 VINTAGE CHRISTMAS NAPKINS,PACK OF 20 SPACEBOY NAPKINS,BAKING SET SPACEBOY DESIGN,EMBROIDERED RIBBON REEL DAISY,...,None,None,None,None,None,None,None,None,None,None
389,TRADITIONAL PICK UP STICKS GAME,GINGERBREAD MAN COOKIE CUTTER,RED RETROSPOT MINI CASES,RED RETROSPOT UMBRELLA,RED RETROSPOT CHILDRENS UMBRELLA,PARISIENNE JEWELLERY DRAWER,FAWN BLUE HOT WATER BOTTLE,TEA PARTY BIRTHDAY CARD,COWBOYS AND INDIANS BIRTHDAY CARD,PENNY FARTHING BIRTHDAY CARD,...,None,None,None,None,None,None,None,None,None,None
390,CAKE STAND WHITE TWO TIER LACE,REGENCY CAKESTAND 3 TIER,POSTAGE,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [19]:
#creating an array list that contains the transacted items
dataset = []
for i in range(len(itemset)):
    dataset.append(itemset.loc[i].dropna().tolist())

In [20]:
#Using the Transaction Encoder to transform the data into the format that can be used
#for Assocication Rule Mining Algorithm
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)
df

,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
0,False,False,False,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
387,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
388,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
389,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
390,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [21]:
#Determine the Frequent_itemset which will be used to determine the Association Rules of the transactions. 
frequent_itemsets = apriori(df, min_support=0.01, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.022959,( DOLLY GIRL BEAKER)
1,0.012755,( I LOVE LONDON MINI BACKPACK)
2,0.017857,( SET 2 TEA TOWELS I LOVE LONDON )
3,0.040816,( SPACEBOY BABY GIFT SET)
4,0.030612,(10 COLOUR SPACEBOY PEN)
...,...,...
39622,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND , RED RETR..."
39623,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND , RED RETR..."
39624,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND , RED RETR..."
39625,0.010204,"(ROUND SNACK BOXES SET OF4 WOODLAND , RED RETR..."


In [22]:
#Generate the Association rules based on the identified Frequent Itemsets
from mlxtend.frequent_patterns import association_rules
association_rules(frequent_itemsets, metric="confidence",min_threshold=0.7)
rules = association_rules(frequent_itemsets, metric="confidence",min_threshold=0.75)
print(rules)

                                              antecedents  \
0                                    ( DOLLY GIRL BEAKER)   
1                                    ( DOLLY GIRL BEAKER)   
2                                    ( DOLLY GIRL BEAKER)   
3                          ( I LOVE LONDON MINI BACKPACK)   
4                          ( I LOVE LONDON MINI BACKPACK)   
...                                                   ...   
721430       (DOLLY GIRL CHILDRENS CUP, JUMBO BAG APPLES)   
721431  (DOLLY GIRL CHILDRENS CUP, ALARM CLOCK BAKELIK...   
721432  (DOLLY GIRL CHILDRENS CUP, SKULL LUNCH BOX WIT...   
721433  (CHILDRENS CUTLERY DOLLY GIRL , SKULL LUNCH BO...   
721434  (LUNCH BOX WITH CUTLERY RETROSPOT , DOLLY GIRL...   

                                              consequents  antecedent support  \
0                             (DOLLY GIRL CHILDRENS BOWL)            0.022959   
1                                               (POSTAGE)            0.022959   
2                       